In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pd

In [10]:
# Transormation (image ==> tensors)
transform=transforms.ToTensor()

In [11]:
# train and test data
train_data=datasets.MNIST(root=r"C:\Users\mgopi\OneDrive\Documents\Dataset",train=True,download=True,transform=transform)
test_data=datasets.MNIST(root=r"C:\Users\mgopi\OneDrive\Documents\Dataset",train=False,download=True,transform=transform)

In [12]:
# load the data batch
train_loader=DataLoader(train_data,batch_size=100,shuffle=True)
test_loader=DataLoader(test_data,batch_size=100,shuffle=False)

In [13]:
# define the model
class CNN (nn.Module):

  def __init__(self):
    super(CNN,self).__init__()
    # define convolution layer
    self.conv1=nn.Conv2d(1,6,3,1)
    self.conv2=nn.Conv2d(6,16,3,1)
    self.conv3=nn.Conv2d(16,32,3,1)

    # fully connected layer
    self.fc1=nn.Linear(32*1*1,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)

  #forward
  def forward(self,x):
    # convolution 1
    x=F.relu(self.conv1(x))
    x=F.max_pool2d(x,2,2)

    # convolution 2
    x=F.relu(self.conv2(x))
    x=F.max_pool2d(x,2,2)

    #convolution 3
    x=F.relu(self.conv3(x))
    x=F.max_pool2d(x,2,2)

    # flattaning
    x=x.view(-1,32*1*1)

    # fully connected layer
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=F.log_softmax(self.fc3(x),dim=1)

    return x

In [14]:
model=CNN()

In [15]:
# loss function and optimizer
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [16]:
#train and test

epochs=5

for i in range (epochs):
  for j,(x_train,y_train) in enumerate(train_loader):


    #train
    y_pred=model(x_train)
    loss=criterion(y_pred,y_train)

    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #print the progress
    if j%100==0:
      print(f"Epoch {i} ==> patch {(j//100)+1} ==> loss {loss.item()}")

with torch.no_grad():
  test_losses=[]
  correct = 0
  total = 0
  for x_test,y_test in test_loader:
    y_pred=model(x_test)
    loss=criterion(y_pred,y_test)
    test_losses.append(loss.item())
    pred=y_pred.argmax(dim=1)
    correct+=(pred==y_test).sum().item()
    total+=y_test.size(0)
  print("Testing".center(80,'-'))
  accuracy=100*correct/total
  loss=sum(test_losses)/len(test_losses)
  print(f"Test Accuracy {accuracy} and Test loss {loss}")

Epoch 0 ==> patch 1 ==> loss 2.2919793128967285
Epoch 0 ==> patch 2 ==> loss 1.1737070083618164
Epoch 0 ==> patch 3 ==> loss 0.5783759355545044
Epoch 0 ==> patch 4 ==> loss 0.408760666847229
Epoch 0 ==> patch 5 ==> loss 0.2335146963596344
Epoch 0 ==> patch 6 ==> loss 0.224348247051239
Epoch 1 ==> patch 1 ==> loss 0.43707016110420227
Epoch 1 ==> patch 2 ==> loss 0.46433013677597046
Epoch 1 ==> patch 3 ==> loss 0.19358626008033752
Epoch 1 ==> patch 4 ==> loss 0.19407311081886292
Epoch 1 ==> patch 5 ==> loss 0.10148365050554276
Epoch 1 ==> patch 6 ==> loss 0.12885338068008423
Epoch 2 ==> patch 1 ==> loss 0.1588183045387268
Epoch 2 ==> patch 2 ==> loss 0.05651653930544853
Epoch 2 ==> patch 3 ==> loss 0.16339614987373352
Epoch 2 ==> patch 4 ==> loss 0.1572973132133484
Epoch 2 ==> patch 5 ==> loss 0.1537092626094818
Epoch 2 ==> patch 6 ==> loss 0.12238281965255737
Epoch 3 ==> patch 1 ==> loss 0.08909334242343903
Epoch 3 ==> patch 2 ==> loss 0.12058959156274796
Epoch 3 ==> patch 3 ==> loss 0.